In [1]:
import pandas as pd
import findspark
findspark.init() 
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName('yellow_data').master('local[4]').config("spark.jars", "/home/faraaz/Downloads/postgresql-42.3.3.jar").getOrCreate()


23/01/19 08:51:30 WARN Utils: Your hostname, faraaz-Lenovo-G50-80 resolves to a loopback address: 127.0.1.1; using 192.168.1.33 instead (on interface wlp3s0)
23/01/19 08:51:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/19 08:51:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#Connection to Postgres

# df = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql://localhost:5431/ny_taxi") \
#     .option("dbtable", "yellow_taxi_data") \
#     .option("user", "root") \
#     .option("password", "root") \
#     .option("driver", "org.postgresql.Driver") \
#     .load()

In [53]:
spark.read.options(inferSchema=True, header=True).csv("/home/faraaz/work_folder/zoomcamp/datasets/yellow_tripdata_2021-01.csv").write.format('parquet').save("/home/faraaz/work_folder/zoomcamp/output/silver/parquet")


In [54]:
df =spark.read.format("parquet").load("/home/faraaz/work_folder/zoomcamp/output/silver/parquet/*")

In [5]:
df.count()

1369765

In [6]:
df.printSchema()    

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [57]:
df = df.withColumn("tpep_pickup_datetime", to_timestamp(col("tpep_pickup_datetime"),'yyyy-MM-dd HH:mm:ss'))\
    .withColumn("tpep_dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime"),'yyyy-MM-dd HH:mm:ss'))

In [58]:
df.write \
    .format("jdbc") \
    .mode("overwrite")\
    .option("url", "jdbc:postgresql://localhost:5431/ny_taxi") \
    .option("dbtable", "yellow_taxi_data") \
    .option("user", "root") \
    .option("password", "root") \
    .option("driver", "org.postgresql.Driver") \
    .save()

In [59]:
spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5431/ny_taxi") \
    .option("dbtable", "yellow_taxi_data") \
    .option("user", "root") \
    .option("password", "root") \
    .option("driver", "org.postgresql.Driver") \
    .load().count()

1369765